<a href="https://colab.research.google.com/github/esserilee/Innopolis/blob/main/%D1%87%D0%B0%D1%82_%D0%B1%D0%BE%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 загружаем модель

In [ ]:
#я выбрала ответы на вопросы
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline
#загружаем модель
model_name = "ai-forever/rugpt3small_based_on_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token #Использование EOS токена как PAD токена
model.config.pad_token_id = tokenizer.eos_token_id





/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

загружаем датасет

In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("darkQibit/russian-spam-detection")
dataset = ds["train"].shuffle(seed=42).select(range(50))  # Опционально, можно отключить при реальном тюнинге
print(dataset[0])


README.md:   0%|          | 0.00/870 [00:00<?, ?B/s]

processed_combined.parquet:   0%|          | 0.00/480M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4511513 [00:00<?, ? examples/s]

{'message': 'Apple aapl временно закрывает сайты в магазины, мэриленде и оттаве apple временно закрыла три магазина на фоне всплеска covid19', 'label': 0}


препроцессинг

In [ ]:
def preprocess_function(examples):
    texts = [f"<s>Сообщение: {m}</s>" for m in examples["message"]]
    tokens = tokenizer(
        texts,
        truncation=True,
        padding="max_length",  # или padding=True
        max_length=128,
        return_tensors=None    # ❗ очень важно
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens
tokenized_ds = dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
def chat(prompt, model, max_new_tokens=20, temperature=0.8):
    input_text = f"Вопрос: {prompt} Ответ:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(model.device)

    output_ids = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    if "Ответ:" in response:
        response = response.split("Ответ:")[-1].strip()
    return response



for q in ["привет", "как дела", "что ты умеешь?"]:
    print(f"👤: {q}")
    print(f"🤖: {chat(q, model)}\n")


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


👤: привет
🤖: Я не вижу смысла в вашей статье.  Вы не можете не понимать, что вы сами пишете

👤: как дела
🤖: Здравствуйте!  Подскажите пожалуйста, как часто я обращаюсь к вам за советом по поводу вопросов о том

👤: что ты умеешь?
🤖: я знаю, как обращаться с компьютером, но я не знаю, как это сделать, если у тебя



Дообучение

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

#указываем аргументы дообучения модели
import os
os.environ["WANDB_DISABLED"] = 'true'

training_args = TrainingArguments(
    output_dir="./rugpt3_chatbot", # Папка для сохранения всех артефактов обучения
    per_device_train_batch_size = 2, # Сколько записей приходится на один батч
    num_train_epochs = 3, # Количество эпох
    learning_rate = 5e-5, # Скорость обучения
    fp16 = torch.cuda.is_available(), # На каком устройстве происходит обучение
    logging_steps = 10, # Через сколько шагов выдается информация об обучении модели
    save_total_limit = 1, # Сколько раз логируется процесс обучения модели
    report_to = "none" # Отключает интеграцию с WANDB, которая с сентября заблокирована в России
)

#дообучение
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

model.save_pretrained("content/my_rugpt_chatbot")
tokenizer.save_pretrained("content/my_rugpt_chatbot")

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "content/my_rugpt_chatbot"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

for q in ["привет", "как дела", "что ты умеешь?"]:
    print(f"👤: {q}")
    print(f"🤖: {chat(q, model)}\n")

/tmp/ipython-input-2782902464.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
10,1.287900
20,1.018000
30,0.607900
40,0.831800
50,0.748800
60,0.412700
70,0.989100


👤: привет
🤖: все приложения после запуска виндовс начинают глючить при запуске игры, и если вы регулярно

👤: как дела
🤖: все в порядке, все в порядке, все в порядке, все в порядке, все в порядке,

👤: что ты умеешь?
🤖: то ли еще будет, когда нибудь это пригодиться. Должно же быть какое то дело, о котором ты



что до обучения, что после выводит бред, но во втором случае уже получше :(
